In [1]:
import numpy as np
from math import factorial
import random
from multiprocessing import Pool
from scipy.stats import poisson
import matplotlib.pyplot as plt


In [2]:
def update_likelihood(rho_previous, new_counts, mu):
    """ at step S the likelihood is the product of poissonian distribution"""
    return rho_previous * poisson.pmf(new_counts,mu)

def get_probabilities(rho_D, rho_B):
    """The probability of being bright is calculated with bayes rule."""
    s = rho_D + rho_B
    return (rho_D/s, rho_B/s)

def get_entropy(PD, PB):
    """returns the expected entropy value given the probability distribution"""
    H = -( PD*np.log(PD) + PB*np.log(PB)) 
    return H

    

In [3]:
def detect_bin(PB, mu_B, mu_D):
    # throw a coin to decide if it is bright or dark according to PB
    print(PB)
    p = random.random()
    if PB<0.5:
        if p<PB: #means the ion is Bright
            return np.random.poisson(mu_B)
        else:
            return np.random.poisson(mu_D)
    else:
        if p<(1-PB):#means the ion is dark
             return np.random.poisson(mu_D)
        else:
            return np.random.poisson(mu_B)


In [4]:

def is_pi_pulse_needed(rho_smin_B, rho_smin_D, mu_B, mu_D):
    """rho_smin_B, rho_smin_D are the likelihood at the previous step"""
    H_diff_tot=0
    max_n=10
    for ns in range(0,max_n):
        # calculate the possible likelihoods if we apply or not a pi pulse
        rho_s_B = update_likelihood(rho_smin_B, ns, mu_B)
        rho_s_D = update_likelihood(rho_smin_D, ns, mu_D)
        
        rho_sPi_B = update_likelihood(rho_smin_B, ns, mu_D) # if we were to apply a pi pulse
        rho_sPi_D = update_likelihood(rho_smin_D, ns, mu_B)
        # calculate the associated probabilities
        (P_s_D, P_s_B) = get_probabilities(rho_s_D, rho_s_B)
        (P_sPi_D, P_sPi_B) = get_probabilities(rho_sPi_D, rho_sPi_B)
        
        #calculate the entropies
        H_s = get_entropy(P_s_D, P_s_B)
        H_sPi = get_entropy(P_sPi_D, P_sPi_B)
        
        H_diff_tot =H_diff_tot + (H_sPi- H_s)*(poisson.pmf(ns,mu_B)+poisson.pmf(ns,mu_D))
#         print(H_diff_tot)
    
    if H_diff_tot > 0:
        return True
    else:
        return False


In [5]:
rho_s_B=1
rho_s_D=1

def detection(PB, optimize, nbins):
    pi_pulse=False
    rho_s_B=1
    rho_s_D=1
    reversed_state=False
    e_c= 0.0001
    n_pi_pulses=0
    
    bin_time = 200./nbins # in us
    mu_B = R_B *bin_time;
    mu_D = R_D *bin_time;

    for s in range(0,nbins):
        counts = int(round(detect_bin(PB, mu_B, mu_D))); #PB probability of being bright 
        print(counts)
        if reversed_state:
            rho_s_B = update_likelihood(rho_s_B, counts, mu_D)
            rho_s_D = update_likelihood(rho_s_D, counts, mu_B)
        else:
            rho_s_B = update_likelihood(rho_s_B, counts, mu_B)
            rho_s_D = update_likelihood(rho_s_D, counts, mu_D)
        # calculate errors
        eB= rho_s_D/(rho_s_B + rho_s_D)
        eD= rho_s_B/(rho_s_B + rho_s_D)
        if min(eB,eD)<e_c:
            bright = 1
            if rho_s_B < rho_s_D:
                bright= 0
            return(s, bright, n_pi_pulses)
        else:
            if optimize:
                need_pi_pulse = is_pi_pulse_needed(rho_s_B, rho_s_D,mu_B, mu_D)
                if(need_pi_pulse):
                    if PB==1:
                        F=1
                        PB= F*(1-PB)
                    PB=(1-PB)
                    n_pi_pulses+=1
                    reversed_state= not reversed_state
                    
    bright = 1
    if rho_s_B < rho_s_D:
        bright = 0
    return (nbins, bright,n_pi_pulses)
        



In [1]:
def repeated_detection(Pb, rep, bins, optimize):
    a_Pb = np.full(rep, Pb)
    a_optimize = np.full(rep, optimize)
    a_bins = np.full(rep, bins)
    with Pool(8) as p:  # Pool(n cores)
        res_list = p.starmap(detection, zip(a_Pb, a_optimize, a_bins))
        
    bins_mean=np.mean(list(zip(*res_list))[0])
    p_mean=np.mean(list(zip(*res_list))[1])
    n_pulses=np.mean(list(zip(*res_list))[2])
    
    return (p_mean,bins_mean,n_pulses)

In [ ]:
repeated_detection(1, 10, 20, True) 

[1 1 1 1 1 1 1 1 1 1]


In [ ]:
rep=100
max_bins = 50
min_bins= 5
det_mean = np.zeros(max_bins-min_bins)
det_t_mean = np.zeros(max_bins-min_bins)
det_n_mean = np.zeros(max_bins-min_bins)
det_mean_std = np.zeros(max_bins-min_bins)
det_t_mean_std = np.zeros(max_bins-min_bins)

det_mean_noPi= np.zeros(max_bins-min_bins)
det_t_mean_noPi = np.zeros(max_bins-min_bins)
det_n_mean_noPi = np.zeros(max_bins-min_bins)
det_mean_std_noPi = np.zeros(max_bins-min_bins)
det_t_mean_std_noPi = np.zeros(max_bins-min_bins)

det_mean_dark = np.zeros(max_bins-min_bins)
det_t_mean_dark = np.zeros(max_bins-min_bins)
det_n_mean_dark = np.zeros(max_bins-min_bins)
det_mean_std_dark = np.zeros(max_bins-min_bins)
det_t_mean_std_dark = np.zeros(max_bins-min_bins)

det_mean_noPi_dark= np.zeros(max_bins-min_bins)
det_t_mean_noPi_dark = np.zeros(max_bins-min_bins)
det_n_mean_noPi_dark = np.zeros(max_bins-min_bins)
det_mean_std_noPi_dark = np.zeros(max_bins-min_bins)
det_t_mean_std_noPi_dark = np.zeros(max_bins-min_bins)

R_B= 25 /200  #Bright state rate counts per 200us detection time
R_D= 0.1 /200


for bins in range(min_bins,max_bins):   
    res = repeated_detection(1, rep, bins, True)     
    det_mean[bins-min_bins] = res[0] 
    det_t_mean[bins-min_bins] = res[1]*200/bins
    det_n_mean[bins-min_bins] = res[2] 
        
    res = repeated_detection(1, rep, bins, False)

    det_mean_noPi[bins-min_bins] = res[0] 
    det_t_mean_noPi[bins-min_bins] = res[1]*200/bins
    det_n_mean_noPi[bins-min_bins] = res[2] 
    
    res = repeated_detection(0, rep, bins, True)     
    det_mean_dark[bins-min_bins] = res[0] 
    det_t_mean_dark[bins-min_bins] = res[1]*200/bins
    det_n_mean_dark[bins-min_bins] = res[2] 
        
    res = repeated_detection(0, rep, bins, False)

    det_mean_noPi_dark[bins-min_bins] = res[0] 
    det_t_mean_noPi_dark[bins-min_bins] = res[1]*200/bins
    det_n_mean_noPi_dark[bins-min_bins] = res[2] 


    

In [ ]:
plt.plot(range(min_bins, max_bins), det_t_mean,range(min_bins, max_bins), det_t_mean_noPi)
bright_mean

In [ ]:
plt.plot(range(min_bins, max_bins), det_t_mean_dark,range(min_bins, max_bins), det_t_mean_noPi_dark)
bright_mean

In [ ]:
plt.plot(range(min_bins, max_bins), t_mean,range(min_bins, max_bins), t_mean_std)

In [ ]:
plt.plot(range(min_bins, max_bins), n_mean)

In [ ]:
np.mean(list(zip(*res_list))[0])
np.mean(list(zip(*res_list))[1])

In [ ]:
pb_mean

In [2]:
pb_mean

NameError: name 'pb_mean' is not defined

In [6]:
mp.cpu_count()

4

In [5]:
import multiprocessing as mp